In [6]:
%matplotlib inline
import matplotlib as plt
import pandas as pd
from datetime import datetime
import geoplotlib
import vincent
from vincent import AxisProperties, ValueRef, PropertySet
vincent.core.initialize_notebook()

In [2]:
from IPython.core.display import HTML
css = open("styles/style-table.css").read() + open('styles/style-notebook.css').read() + open("styles/test.css").read()
HTML('<style>{}</style>'.format(css))

IndentationError: unexpected indent (<ipython-input-53-53c7c7358822>, line 2)

In [98]:
#  Let's visualize the data we have with a barchart
tbp_geo_data = pd.read_csv("/Users/coreypetty/PycharmProjects/TBP_metrics/raw_data/countries_2016-04-04-clean.csv")
# tbp_geo_data = tbp_geo_data.set_index('country_name')
vis = vincent.Bar(tbp_geo_data, columns=['total_downloads'], height=400, width=1500)
vis.axes[0].properties = AxisProperties(
    labels=PropertySet(
        angle=ValueRef(value=90),
        align=ValueRef(value='left')
        )
    )
# geo_data.head()
vis.display()

In [80]:
world_topo = "geo_data/vincent_map_data/world-countries.topo.json"
geo_data = [{'name': 'countries',
             'url': world_topo,
             'feature': 'world-countries'}]

vis = vincent.Map(data=tbp_geo_data, geo_data=geo_data, scale=200, 
          map_key={'country_name': 'total_downloads'})
# vis.marks[0].properties.enter.stroke_opacity = ValueRef(value=0.5)
# vis.to_json('vega.json')
vis.display()

In [9]:
#Map Data Binding
import json
import pandas as pd

In [35]:
#Map the county codes we have in our geometry to those in the
#county_data file, which contains additional rows we don't need
with open('geo_data/vincent_map_data/us_counties.topo.json', 'r') as f:
    get_id = json.load(f)

#A little FIPS code munging
new_geoms = []
for geom in get_id['objects']['us_counties.geo']['geometries']:
    geom['properties']['FIPS'] = int(geom['properties']['FIPS'])
    new_geoms.append(geom)

get_id['objects']['us_counties.geo']['geometries'] = new_geoms

with open('geo_data/vincent_map_data/us_counties.topo.json', 'w') as f:
    json.dump(get_id, f)

#Grab the FIPS codes and load them into a dataframe
geometries = get_id['objects']['us_counties.geo']['geometries']
county_codes = [x['properties']['FIPS'] for x in geometries]
county_df = pd.DataFrame({'FIPS': county_codes}, dtype=str)
county_df = county_df.astype(int)

#Read into Dataframe, cast to string for consistency
# df = pd.read_csv('data/us_county_data.csv', na_values=[' '])
# df['FIPS_Code'] = df['FIPS'].astype(str)

#Perform an inner join, pad NA's with data from nearest county
# merged = pd.merge(df, county_df, on='FIPS', how='inner')
# merged = merged.fillna(method='pad')

In [70]:
get_id['objects']['us_counties.geo']['geometries'][0]

{'arcs': [[[0, 1, 2, 3, -4, 4, 5, 6, 7, -8, 8, 9]],
  [[10, 11, 12, 13, 14, 15, 16, 17, 18]]],
 'properties': {'FIPS': 22051},
 'type': 'MultiPolygon'}

In [72]:
#Map the county codes we have in our geometry to those in the
#county_data file, which contains additional rows we don't need
with open('geo_data/vincent_map_data/world-countries.topo.json', 'r') as f:
    get_id_world = json.load(f)

In [73]:
get_id_world['objects']['world-countries']['geometries'][0]

{'arcs': [[0, 1, 2, 3, 4, 5]],
 'id': 'AFG',
 'properties': {'name': 'Afghanistan'},
 'type': 'Polygon'}

In [79]:
#A little FIPS code munging
new_geoms = []
for geom in get_id_world['objects']['world-countries']['geometries']:
    geom['properties']['name'] = str(geom['properties']['name'])
    new_geoms.append(geom)
    
get_id_world['objects']['world-countries']['geometries'] = new_geoms

geometries = get_id_world['objects']['world-countries']['geometries']

In [93]:
import plotly.tools as tls
tls.set_credentials_file(username='CoreyPetty', api_key='lhkmbpdh7g')

In [118]:
import plotly.plotly as py
import pandas as pd

data = [ dict(
        type = 'choropleth',
        locations = locations,
        z = tbp_geo_data['total_downloads'],
        text = tbp_geo_data['country_name'],
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            )
        ),
        zmin = 0,
        colorbar = dict(
            tickprefix = '',
            title = 'Total<br>downloads'
        ),
    ) ]

layout = dict(
    title = 'The Bitcoin Podcast global downloads',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.plot(fig)

'https://plot.ly/~CoreyPetty/10'

In [104]:
import pycountry

In [114]:
mapping = {country.name: country.alpha3 for country in pycountry.countries}
# for column in tbp_geo_data:
#     print column['name'], mapping.get(column['name'], 'No country found')

In [116]:
locations = []
for country in tbp_geo_data['country_name']:
    locations.append(mapping.get(country,"not found"))

In [119]:
tbp_geo_data = pd.read_csv("/Users/coreypetty/PycharmProjects/TBP_metrics/raw_data/countries_2016-04-04-noUS.csv")
data = [ dict(
        type = 'choropleth',
        locations = locations,
        z = tbp_geo_data['total_downloads'],
        text = tbp_geo_data['country_name'],
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            )
        ),
        zmin = 0,
        colorbar = dict(
            tickprefix = '',
            title = 'Total<br>downloads'
        ),
    ) ]

layout = dict(
    title = 'The Bitcoin Podcast global downloads',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.plot(fig)

'https://plot.ly/~CoreyPetty/12'